In [1]:
import time
import os
import torch
from sbi import utils as utils
from sbi.inference import SNPE, SNPE_A, SNPE_C
from sbi.utils.user_input_checks import process_prior

from parsing_functions import save_pickle_data, load_inference_data, extract_specs, preprocess

## Loading dataset

In [2]:
save_dir = 'saved_datasets/model2/'
dataset_name = 'dataset_10000sim_5e+04np_1e-02dt_5os_1e+03pre_1.pickle'

dataset_path = os.path.join(save_dir, dataset_name)

xstack, theta = load_inference_data(input_file = dataset_path)

print(xstack.shape)
features = ['Cxx','Cyy','S_red_x', 'S_red_y']
x = preprocess(data = xstack, features = features ,density = 1.)

print(type(x), type(xstack))

# Extract the specifications
specs_dict = extract_specs(dataset_path)

# Unpack the specifications from the dictionary
num_simulations = specs_dict['num_simulations']
Npts = specs_dict['Npts']
dt = specs_dict['dt']
oversampling = specs_dict['oversampling']
prerun = specs_dict['prerun']

prefix = specs_dict['prefix']
data_type = specs_dict['data_type']

high_tensor = specs_dict['high_tensor']
low_tensor = specs_dict['low_tensor']

# Declaring SBI prior for sampling
prior_sbi = utils.BoxUniform(low=low_tensor, high=high_tensor)

# Check prior, return PyTorch prior.
prior, num_parameters, prior_returns_numpy = process_prior(prior_sbi)

torch.Size([10000, 4, 1000])
<class 'torch.Tensor'> <class 'torch.Tensor'>


## Inference

In [3]:
inference = SNPE(prior=prior)
inference = inference.append_simulations(theta, x)
start = time.time()
density_estimator = inference.train()
posterior = inference.build_posterior(density_estimator)
end = time.time()
print('\n time passed for convergence: ', end-start)

 Neural network successfully converged after 62 epochs.
 time passed for convergence:  199.89245200157166


In [5]:
#saving generated posterior from SBI
posterior_dir = 'saved_posteriors/model2/'

data_posterior = {
    'posterior': posterior,
    'num_simulations': num_simulations,
    'Npts': Npts,
    'dt': dt,
    'oversampling': oversampling,
    'prerun': prerun,
    'low_tensor': low_tensor,
    'high_tensor': high_tensor,
    'data_type': data_type, # Indicates the type of the data
    'features': features
}

save_pickle_data(data=data_posterior, folder_path=posterior_dir)

Saved dataset at saved_posteriors/model2/posterior_10000sim_5e+04np_1e-02dt_5os_1e+03pre.pickle



'saved_posteriors/model2/posterior_10000sim_5e+04np_1e-02dt_5os_1e+03pre.pickle'